 3.环境RNA校正（可选）
- 3.1 准备R环境
- 3.2 创建副本、移位对数归一化
- 3.3 邻域图、莱顿聚类
- 3.4 准备数据传递给R
- 3.5 准备环境RNA校正所需的原始基因矩阵
- 3.6 调用R环境中的SoupX
- 3.7 整合环境RNA校正结果
- 3.8 基础过滤
-------------------------------
SoupX 背景矫正
- 可以在没有聚类信息的情况下运行出
- 如果提供基本聚类结果会更好
- SoupX的结果对所使用的聚类类型并不强烈敏感。

In [1]:
## 3.1 准备R环境
import anndata2ri
import logging 

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.CRITICAL) # 
ro.pandas2ri.activate() # type: ignore
anndata2ri.activate()

%load_ext rpy2.ipython 

/tmp/ipykernel_3183318/1680916089.py:10: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [25]:
## 3.2 读取数据、创建副本、移位对数归一化
adata = sc.read_h5ad("anndata_qc.h5ad")

adata_soup = adata.copy()  # backup
sc.pp.normalize_per_cell(adata_soup)
sc.pp.log1p(adata_soup)

normalizing by total count per cell


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


In [26]:
## 3.3 邻域图、莱顿聚类
"""soupx可以不聚类,聚类后效果更好"""
sc.pp.pca(adata_soup) # add obsm
sc.pp.neighbors(adata_soup)# add varm
sc.tl.leiden(adata_soup, key_added="soupx_groups") # add obs

# Preprocess variables for SoupX
soupx_groups = adata_soup.obs["soupx_groups"]
del adata_soup

computing PCA
    with n_comps=50
    finished (0:00:21)
    and added
    'X_pca', the PCA coordinates (adata.obs)
    'PC1', 'PC2', ..., the loadings (adata.var)
    'pca_variance', the variance / eigenvalues (adata.uns)
    'pca_variance_ratio', the variance ratio (adata.uns)
computing neighbors
    computing neighbors
    using 'X_pca' with n_pcs = 50


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-16 10:35:51.162500: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-16 10:35:51.186718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 10:35:51.363008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the

    computed neighbors (0:00:20)
    computed connectivities (0:00:02)
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:23)
running Leiden clustering
    finished: found 42 clusters and added
    'soupx_groups', the cluster labels (adata.obs, categorical) (0:00:12)


In [27]:
## 3.4 准备数据传递给R
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

In [28]:
## 3.5 准备环境RNA校正所需的原始基因矩阵
## ：是原始矩阵在cellranger的输出中三合一
adata_raw = sc.read_h5ad("./anndata_raw.h5ad")
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T

del adata_raw

In [29]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 
## 3.6 调用R环境中的SoupX
library(SoupX)
# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, were retired in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
2932 genes passed tf-idf cut-off and 231 soup quantile filter.  Taking the top 100.
Using 2049 independent estimates of rho.
Estimated global rho of 0.01
Expanding counts from 42 clusters to 26061 cells.
In addition: Warning message:
In sparseMatrix(i = out@i[w] + 1, j = out@j[w] + 1, x = out@x[w],  :
  'giveCsparse' is deprecated; setting repr="T" for you


In [31]:
## 3.7 整合环境RNA校正结果
adata.layers["counts"] = adata.X
adata.layers["soupX_counts"] = out.T
adata.X = adata.layers["soupX_counts"]

In [32]:
## 3.8 基础过滤
print(f"Total number of genes: {adata.n_vars}")
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
print(f"Number of genes after cell filter: {adata.n_vars}")

Total number of genes: 22164
filtered out 3 cells that have less than 200 genes expressed


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


filtered out 3972 genes that are detected in less than 3 cells


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Number of genes after cell filter: 18192


 4.双峰检测
- 4.1 双连体过滤
- 4.2 保存

In [33]:
# 4.1 双连体过滤
# obs 新增scrublet_score 和 predicted_doublet
sc.external.pp.scrublet(adata, random_state=123)
# 新增uns.scrublet
adata = adata[adata.obs["predicted_doublet"] == False, :]

Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.72
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 4.6%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 0.6%
    Scrublet finished (0:00:37)


/home/fanxi/mambaforge/envs/lang/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [34]:
# 4.2 保存
adata.write("anndata_scrublet.h5ad", compression="gzip") # type: ignore